# Chapter 18. Advanced Time Series Topics  
[Home](http://solomonegash.com/) | [Stata](http://solomonegash.com/woodridge1/index.html) | [R](http://solomonegash.com/econometrics/rbook1/index.html)


In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss

import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

from wooldridge import *

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Example 18.1. Housing Investment and Residential Price Inflation

In [3]:
ts_df = dataWoo('hseinv')

In [4]:
reg1 = smf.ols(formula='linvpc ~ t', data=ts_df).fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:                 linvpc   R-squared:                       0.335
Model:                            OLS   Adj. R-squared:                  0.319
Method:                 Least Squares   F-statistic:                     20.19
Date:                Tue, 02 Jul 2024   Prob (F-statistic):           5.85e-05
Time:                        18:46:05   Log-Likelihood:                 23.290
No. Observations:                  42   AIC:                            -42.58
Df Residuals:                      40   BIC:                            -39.11
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8413      0.045    -18.802      0.0

In [5]:
ts_df['u'] = reg1.resid
RationalDL = smf.ols(formula='u ~ gprice + u.shift(1) + gprice.shift(1)', data=ts_df).fit()
GeometricDL = smf.ols(formula='u ~ gprice + u.shift(1)', data=ts_df).fit()

results_table = summary_col([GeometricDL, RationalDL], 
                            stars=True, 
                            model_names=['GeometricDL', 'RationalDL'], 
                            info_dict={'N':lambda x: f"{int(x.nobs)}"})

results_table.title = "Table 18.1 Distributed Lag Models for Housing Investment: log(invpc)"
print(results_table)

Table 18.1 Distributed Lag Models for Housing Investment: log(invpc)
                GeometricDL RationalDL
--------------------------------------
Intercept       -0.0100     0.0059    
                (0.0179)    (0.0169)  
R-squared       0.4067      0.5420    
R-squared Adj.  0.3754      0.5038    
gprice          3.0948***   3.2564*** 
                (0.9333)    (0.9703)  
gprice.shift(1)             -2.9363***
                            (0.9732)  
u.shift(1)      0.3399**    0.5472*** 
                (0.1316)    (0.1517)  
N               41          40        
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


### Example 18.2. Unit Root Test for Three-Month T-Bill Rates

In [7]:
ts_df = dataWoo('intqrt')
print(ts_df.columns.tolist())

['r3', 'r6', 'r12', 'p3', 'p6', 'hy6', 'hy3', 'spr63', 'hy3_1', 'hy6_1', 'spr63_1', 'hy6hy3_1', 'cr3', 'r3_1', 'chy6', 'chy3', 'chy6_1', 'chy3_1', 'cr6', 'cr6_1', 'cr3_1', 'r6_1', 'cspr63']


In [8]:
uroot1 = smf.ols(formula='cr3 ~ r3.shift(1)', data=ts_df).fit()
rho = 1 + uroot1.params[1]
print(rho)


0.9092893810799101


In [17]:
uroot2 = smf.ols(formula='r3 ~ r3.shift(1)', data=ts_df).fit()
print(summary_col([uroot1, uroot2], 
                            stars=True, 
                            model_names=['uroot1', 'uroot2'], 
                            info_dict={'N':lambda x: f"{int(x.nobs)}"}
                 )
     )


                 uroot1    uroot2 
----------------------------------
Intercept      0.6253**  0.6253** 
               (0.2608)  (0.2608) 
r3.shift(1)    -0.0907** 0.9093***
               (0.0367)  (0.0367) 
R-squared      0.0481    0.8355   
R-squared Adj. 0.0403    0.8341   
N              123       123      
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


### Example 18.3. Unit Root Test for Annual U.S. Inflation

In [12]:
df = dataWoo('phillips')
tsphillips = df[df['year'] < 1997]

uroot3 = smf.ols(formula='cinf ~ inf_1 + cinf.shift(1)', data=tsphillips).fit()
rho1 = 1 + uroot3.params[1]
print(rho1)

0.6896747685117395


In [13]:
reg2 = smf.ols(formula='cinf ~ inf.shift(1)', data=tsphillips).fit()
rho2 = 1 + reg2.params[1]
print(rho2)

0.6652585944080388


In [16]:
print(summary_col([uroot3, reg2], 
                            stars=True, 
                            model_names=['uroot3', 'reg2'], 
                            info_dict={'N':lambda x: f"{int(x.nobs)}"}))


                 uroot3      reg2   
------------------------------------
Intercept      1.3608**   1.2767**  
               (0.5167)   (0.5577)  
R-squared      0.1719     0.1755    
R-squared Adj. 0.1343     0.1576    
cinf.shift(1)  0.1384               
               (0.1264)             
inf.shift(1)              -0.3347***
                          (0.1070)  
inf_1          -0.3103***           
               (0.1027)             
N              47         48        
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


### Example 18.4. Unit Root in the Log of U.S. Real Gross Domestic Product

In [44]:
df = dataWoo('inven')
df['trend'] = range(1, len(df) + 1) #Add trend varaible
df['lngdp']=np.log(df['gdp'])
df.head(10)

,year,i3,inf,inven,gdp,r3,cinven,cgdp,cr3,ci3,cinf,ginven,ggdp,trend,lngdp
0,1959,3.41,0.7,401.399994,2212.300049,2.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,7.701788
1,1960,2.93,1.7,412.000000,2261.699951,1.23,10.600006,49.399902,-1.48,-0.48,1.0,0.026065,0.022084,2,7.723872
2,1961,2.38,1.0,420.899994,2309.800049,1.38,8.899994,48.100098,0.15,-0.55,-0.7,0.021372,0.021044,3,7.744916
3,1962,2.78,1.0,440.899994,2449.100098,1.78,20.000000,139.300049,0.40,0.40,0.0,0.046423,0.058560,4,7.803476
4,1963,3.16,1.3,459.000000,2554.000000,1.86,18.100006,104.899902,0.08,0.38,0.3,0.040232,0.041940,5,7.845416
5,1964,3.55,1.3,474.700012,2702.899902,2.25,15.700012,148.899902,0.39,0.39,0.0,0.033633,0.056664,6,7.902081
6,1965,3.95,1.6,504.799988,2874.800049,2.35,30.099976,171.900146,0.10,0.40,0.3,0.061479,0.061658,7,7.963738
7,1966,4.88,2.9,547.200012,3060.199951,1.98,42.400024,185.399902,-0.37,0.93,1.3,0.080652,0.062497,8,8.026236
8,1967,4.32,3.1,579.200012,3140.199951,1.22,32.000000,80.000000,-0.76,-0.56,0.2,0.056833,0.025806,9,8.052042
9,1968,5.34,4.2,606.099976,3288.600098,1.14,26.899963,148.400146,-0.08,1.02,1.1,0.045397,0.046175,10,8.098217


In [45]:
gdp_reg = smf.ols(formula='ggdp ~ trend + lngdp.shift(1) + ggdp.shift(1)', data=df).fit()
rho_g1 = 1 + gdp_reg.params[2]
print(rho_g1)

0.7903791568083864


In [46]:
gdp_reg2 = smf.ols(formula='ggdp ~ lngdp.shift(1) + ggdp.shift(1)', data=df).fit()
rho_g2 = 1 + gdp_reg2.params[1]
print(rho_g2)

0.9773124588631911


In [47]:
print(summary_col([gdp_reg, gdp_reg2], 
                            stars=True, 
                            model_names=['gdp_reg', 'gdp_reg2'], 
                            info_dict={'N':lambda x: f"{int(x.nobs)}"}))


                gdp_reg  gdp_reg2
---------------------------------
Intercept      1.6509**  0.2149**
               (0.6664)  (0.1005)
R-squared      0.2680    0.1561  
R-squared Adj. 0.1972    0.1033  
ggdp.shift(1)  0.2638    0.1672  
               (0.1647)  (0.1677)
lngdp.shift(1) -0.2096** -0.0227*
               (0.0866)  (0.0119)
trend          0.0059**          
               (0.0027)          
N              35        35      
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


### Example 18.5. Cointegration between Fertility and Personal Exemption